In [3]:
import re

In [4]:
def parse_cycle(cycle):
    """Parse a cycle notation string and return a dictionary mapping."""
    if not cycle:
        return {}

    cycle = cycle.strip()
    if cycle[0] != '(' or cycle[-1] != ')':
        raise ValueError("Invalid cycle notation")
    
    subcycles = re.findall("\(\w*\)", cycle)
    
    mapping = {}
    for subcycle in subcycles:
    
        elements = subcycle[1:-1]
        
        for i in range(len(elements)):
            start = elements[i]
            end = elements[(i + 1) % len(elements)]
            mapping[start] = end

    return mapping

def cycle_notation_from_mapping(mapping):
    """Convert a mapping to cycle notation."""
    cycles = []
    visited = set()

    alphabet = sorted(set(mapping.keys()).union(set(mapping.values())))

    for letter in alphabet:
        if letter not in visited:
            cycle = []
            current = letter
            while current not in visited:
                cycle.append(current)
                visited.add(current)
                if current in mapping:
                    current = mapping[current]

            if len(cycle) > 1:
                cycles.append(tuple(cycle))

    out = ''.join(f"({''.join(map(str, cycle))})" for cycle in cycles)
    if out == '': out = '()'
    return out

class Permutation:
    def __init__(self, cycle=None):
        if cycle is None:
            cycle = ''
        if type(cycle) is str:
            self.cycle = cycle
            self.mapping = parse_cycle(cycle)
        else:
            raise TypeError("Permutation must be string in cycle notation")

    def __mul__(self, other):
        if type(other) is not Permutation:
            if type(other) == str:
                return self * Permutation(other)
            return NotImplemented
        composition = {}
        alphabet = sorted(set([elem for sublist in list(self.mapping.items()) + list(other.mapping.items()) for elem in sublist]))
        for letter in alphabet:
            current = letter
            if current in self.mapping:
                current = self.mapping[current]
    
            if current in other.mapping:
                composition[letter] = other.mapping[current]
            else:
                composition[letter] = current
        return Permutation(cycle_notation_from_mapping(composition))

    def __pow__(self, exp):
        if exp >= 0:
            out = Permutation()
            for i in range(exp):
                out *= self
            return out
        elif exp == -1:
            return Permutation(cycle_notation_from_mapping({y: x for x, y in self.mapping.items()}))
        elif exp < -1:
            return (self ** -1) ** -exp

    def __repr__(self):
        return self.cycle

    def __hash__(self):
        return hash(self.cycle)

    def __eq__(self, other):
        return self.__hash__() == other.__hash__()

    def __lt__(self, other):
        if len(self.cycle) == len(other.cycle):
            if self.cycle.index(')') == other.cycle.index(')'):
                return self.cycle < other.cycle
            return self.cycle.index(')') < other.cycle.index(')')
        return len(self.cycle) < len(other.cycle)

In [6]:
s_6 = set()
p = Permutation('(12345)')
q = Permutation('(12)')
p_prime = Permutation('(bcdef)')
q_prime = Permutation('(ad)(bc)(ef)')
done = False
s_6.add(p)
s_6.add(q)
mapping = {p:p_prime, q:q_prime}
while not done:
    add = set()
    for perm in s_6:
        perm_t = mapping[perm]
        perm_a = perm * p
        perm_b = perm * q
        if perm_a not in s_6: 
            add.add(perm_a)
            mapping[perm_a] = perm_t * p_prime
        if perm_b not in s_6: 
            add.add(perm_b)
            mapping[perm_b] = perm_t * q_prime
    if len(add) == 0: done = True
    s_6 = s_6.union(add)

In [8]:
mapping

{(12345): (bcdef),
 (12): (ad)(bc)(ef),
 (13524): (bdfce),
 (2345): (adfc),
 (1345): (aebd),
 (): (),
 (14)(235): (afbecd),
 (13452): (afecb),
 (124)(35): (aefdbc),
 (14253): (becfd),
 (135)(24): (adecfb),
 (13)(254): (afebdc),
 (24)(35): (af)(cd),
 (15432): (bfedc),
 (14)(253): (adcebf),
 (14)(35): (ab)(de),
 (153)(24): (abfced),
 (14235): (aebfc),
 (143)(25): (aedfcb),
 (1543): (adbe),
 (15243): (afdec),
 (1542): (aecf),
 (1532): (afdb),
 (14325): (afbde),
 (12543): (abcef),
 (1432): (abec),
 (13254): (aecdb),
 (2543): (acfd),
 (15324): (acfbe),
 (15423): (acdfb),
 (142)(35): (acbdfe),
 (132): (acf)(bde),
 (152): (abe)(cdf),
 (254): (ade)(bfc),
 (15): (ac)(bf)(de),
 (143): (acd)(bfe),
 (253): (abc)(dfe),
 (15)(243): (aebcdf),
 (243): (adb)(cef),
 (154)(23): (abdefc),
 (34): (af)(bc)(de),
 (154): (afd)(bce),
 (142): (afb)(ced),
 (23): (ae)(bf)(cd),
 (45): (ab)(cd)(ef),
 (153): (aef)(bdc),
 (145): (adf)(bec),
 (13): (ab)(ce)(df),
 (24): (ac)(be)(df),
 (12)(45): (ac)(bd),
 (1253): (acde

In [9]:
s_6 = set()
generators = [
    Permutation('(12)'),
    Permutation('(13)'),
    Permutation('(14)'),
    Permutation('(15)'),
    Permutation('(16)')
]

mapping = {
    Permutation('(12)'): Permutation('(12)(34)(56)'),
    Permutation('(13)'): Permutation('(16)(24)(35)'),
    Permutation('(14)'): Permutation('(15)(23)(46)'),
    Permutation('(15)'): Permutation('(14)(25)(36)'),
    Permutation('(16)'): Permutation('(13)(26)(45)'),
}
    
done = False
s_6.update(generators)
while not done:
    add = set()
    for perm in s_6:
        perm_t = mapping[perm]
        for g in generators:
            perm_a = perm * g
            if perm_a not in s_6: 
                add.add(perm_a)
                mapping[perm_a] = perm_t * mapping[g]
    if len(add) == 0: done = True
    s_6 = s_6.union(add)

In [12]:
counts = {}
for p in s_6:
    new_mapping = {k: p * v * p**-1 for k, v in mapping.items()}
    fixed_points = {k: v for k, v in new_mapping.items() if v == k}
    n = len(fixed_points)
    counts[n] = counts.get(n, 0) + 1
    if n == 20:
        print(p, new_mapping['(12)'])
counts

(146) (13)(26)(45)
(132)(456) (16)(23)(45)
(12)(56) (12)(34)(56)
(15462) (14)(26)(35)
(142)(365) (15)(24)(36)
(15236) (13)(24)(56)
(125)(364) (15)(23)(46)
(23456) (16)(23)(45)
(263) (13)(25)(46)
(12)(34) (12)(34)(56)
(1465)(23) (12)(35)(46)
(13624) (12)(35)(46)
(15324) (16)(25)(34)
(36)(45) (12)(34)(56)
(16245) (14)(23)(56)
(14)(2563) (16)(25)(34)
(1346)(25) (13)(24)(56)
(1356)(24) (12)(35)(46)
(24365) (15)(24)(36)
(13)(2654) (15)(26)(34)
(12463) (13)(26)(45)
(16)(2543) (12)(36)(45)
(15264) (12)(36)(45)
(16253) (12)(36)(45)
(12354) (14)(25)(36)
(126)(345) (16)(24)(35)
(16534) (16)(24)(35)
(15643) (15)(23)(46)
(15)(2634) (12)(36)(45)
(16423) (15)(26)(34)
(35)(46) (12)(34)(56)
(16352) (13)(25)(46)
(14523) (12)(35)(46)
(135) (14)(25)(36)
(1435)(26) (14)(23)(56)
(254) (14)(26)(35)


{4: 540, 5: 144, 20: 36}

In [17]:
p = Permutation('(12)(34)')
new_mapping = {k: p * v * p**-1 for k, v in mapping.items()}
fixed_points = {k: v for k, v in new_mapping.items() if v == k}
{k:v for k, v in sorted(fixed_points.items())}

{(): (),
 (2345): (2345),
 (2356): (2356),
 (2436): (2436),
 (2465): (2465),
 (2543): (2543),
 (2564): (2564),
 (2634): (2634),
 (2653): (2653),
 (3546): (3546),
 (3645): (3645),
 (23654): (23654),
 (24563): (24563),
 (25346): (25346),
 (26435): (26435),
 (23)(46): (23)(46),
 (24)(35): (24)(35),
 (25)(36): (25)(36),
 (26)(45): (26)(45),
 (34)(56): (34)(56)}

In [16]:
{k:v for k, v in sorted(new_mapping.items())}

{(): (),
 (12): (12)(34)(56),
 (13): (13)(26)(45),
 (14): (14)(25)(36),
 (15): (15)(23)(46),
 (16): (16)(24)(35),
 (23): (15)(24)(36),
 (24): (16)(23)(45),
 (25): (14)(26)(35),
 (26): (13)(25)(46),
 (34): (12)(35)(46),
 (35): (16)(25)(34),
 (36): (14)(23)(56),
 (45): (13)(24)(56),
 (46): (15)(26)(34),
 (56): (12)(36)(45),
 (123): (164)(235),
 (124): (153)(246),
 (125): (136)(254),
 (126): (145)(263),
 (132): (146)(253),
 (134): (165)(234),
 (135): (124)(356),
 (136): (152)(364),
 (142): (135)(264),
 (143): (156)(243),
 (145): (162)(345),
 (146): (123)(465),
 (152): (163)(245),
 (153): (142)(365),
 (154): (126)(354),
 (156): (134)(256),
 (162): (154)(236),
 (163): (125)(346),
 (164): (132)(456),
 (165): (143)(265),
 (234): (143)(256),
 (235): (132)(465),
 (236): (126)(345),
 (243): (134)(265),
 (245): (125)(364),
 (246): (142)(356),
 (253): (123)(456),
 (254): (152)(346),
 (256): (165)(243),
 (263): (162)(354),
 (264): (124)(365),
 (265): (156)(234),
 (345): (154)(263),
 (346): (136)(24